In [ ]:
from terminusdb_client import WOQLQuery as WQ
from terminusdb_client import WOQLClient

server_url = "https://127.0.0.1:6363"
user = "admin"
account = "admin"
key = "root"
dbid = "Bank_Balance_Example"
repository = "local"
label = "Bank Balance Example"
description = "An example database for playing with bank accounts"

client = WOQLClient(server_url)
result = client.connect(user=user,account="admin",key=key,db=dbid)

In [ ]:
# Uncomment this to delete your old database first
# client.delete_database(dbid)

In [ ]:
# Create the database
try:
    client.create_database(dbid,user,label=label, description=description)
except Exception as E:
    error_obj = E.errorObj
    if "api:DatabaseAlreadyExists" == error_obj.get("api:error",{}).get("@type",None):
        print(f'Warning: Database "{dbid}" already exists!\n')
    else:
        raise(E)

In [ ]:
# Add the schema (there is no harm in adding repeatedly as it is idempotent)
query = WQ().woql_and(
    WQ().doctype("scm:BankAccount")
        .label("Bank Account")
        .description("A bank account")
        .property("scm:owner", "xsd:string")
            .label("owner")
            .cardinality(1)
        .property("scm:balance","xsd:nonNegativeInteger")
            .label("owner")
).execute(client, "Adding bank account object to schema")

In [ ]:
# Fix bug in schema
query = WQ().woql_and(
    WQ().delete_quad("scm:balance", "label", "owner", "schema/main"),
    WQ().add_quad("scm:balance", "label", "balance", "schema/main")
).execute(client, "Label for balance was wrong")

In [ ]:
# Add the data to the main branch
query = WQ().woql_and(
    WQ().insert("doc:001234", "scm:BankAccount")
        .property("scm:owner", "Mike")
        .property("scm:balance", 123)
).execute(client, "Add mike")

In [ ]:
# try to make mike go below zero
account_id, balance, new_balance = WQ().vars("AccountID", "Balance", "New Balance")
try:
    query = WQ().woql_and(
        WQ().triple(account_id, "scm:owner", "Mike"),
        WQ().triple(account_id, "scm:balance", balance),
        WQ().delete_triple(account_id, "scm:balance", balance),
        WQ().eval(WQ().minus(balance, 130), new_balance),
        WQ().add_triple(account_id, "scm:balance", new_balance)
    ).execute(client, "Update mike")
except Exception as E:
    error_obj = E.errorObj
    print(f'Witness of failure for mike: {error_obj}')

In [ ]:
# Subtract less
query = WQ().woql_and(
    WQ().triple(account_id, "scm:owner", "Mike"),
    WQ().triple(account_id, "scm:balance", balance),
    WQ().delete_triple(account_id, "scm:balance", balance),
    WQ().eval(WQ().minus(balance, 110), new_balance),
    WQ().add_triple(account_id, "scm:balance", new_balance)
).execute(client, "Update mike")

In [ ]:
# Mike only has 13 money
(WQ().triple(account_id, "scm:owner", "Mike") 
 + WQ().triple(account_id, "scm:balance", balance)).execute(client)

In [ ]:
# Make the "branch_office" branch
branch = "branch_office"
try:
    client.branch(branch)
except Exception as E:
    error_obj = E.errorObj
    if "api:BranchExistsError" == error_obj.get("api:error",{}).get("@type",None):
        print(f'Warning: Branch "{branch}" already exists!\n')
    else:
        raise(E)

In [ ]:
# Add some data to the branch
client.checkout(branch)
query = WQ().woql_and(
  WQ().insert("doc:034521", "scm:BankAccount")
      .property("owner", "Jim")
      .property("balance", 8)
).execute(client,"Adding Jim")

In [ ]:
# Return to the 'main' branch and add Jane
client.checkout('main')
query = WQ().woql_and(
  WQ().insert("doc:098765", "scm:BankAccount")
      .property("owner", "Jane")
      .property("balance", 887)
).execute(client,"Adding Jane")

In [ ]:
try:
    client.rebase({"rebase_from": f'{user}/{dbid}/{repository}/branch/{branch}',
                   "author": user,
                   "message": "Merging jim in from branch_office"})
except Exception as E:
    error_obj = E.errorObj
    print(f'{error_obj}\n')

In [ ]:
q = WQ().limit(10).triple("v:X","v:Y","v:Z")
client.query(q)